# 04 MODEL TRAINING

In [269]:
import os
import pandas as pd
import numpy as np

In [270]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Get in the correct working directory

In [271]:
%pwd

'/Users/chrissunderland/Desktop/starbucks_store_predictions'

In [5]:
os.chdir("../")

In [6]:
%pwd

'/Users/chrissunderland/Desktop/starbucks_store_predictions'

### Model Experiments

In [277]:
from src.StarbucksProject.constants import *
from src.StarbucksProject.utils.common import read_yaml, create_directories

#### Load the data

In [278]:
X_train_path = read_yaml(CONFIG_FILE_PATH).model_training.x_train_path
y_train_path = read_yaml(CONFIG_FILE_PATH).model_training.y_train_path

[2024-07-12 13:58:53,447: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-07-12 13:58:53,452: INFO: common: yaml file: config/config.yaml loaded successfully]


In [279]:
X_train = pd.read_csv(X_train_path)
y_train = pd.read_csv(y_train_path).values

Complete any remaining data preprocessing steps

In [280]:
from sklearn.preprocessing import StandardScaler

In [281]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#### Fit several models to the data and evaluate baseline performance

In [282]:
from sklearn.model_selection import cross_val_score

In [283]:
def fit_model(model_name, x, y):
    
    model = model_name
    model.fit(x, y)
    
    cv_scores = cross_val_score(model_name, x, y, cv=3, scoring='neg_mean_absolute_error')
    cv_mae_scores = np.abs(cv_scores)

    return cv_mae_scores

#### Model 1 -  K Neighbors Regressor

In [284]:
from sklearn.neighbors import KNeighborsRegressor

In [285]:
knr_mae_scores = fit_model(KNeighborsRegressor(), X_train_scaled, y_train)
print(f"K Neighbors CV MAE scores = {knr_mae_scores}")

K Neighbors CV MAE scores = [0.38685854 0.37168293 0.39392683]


#### Model 2 - Random Forest Regressor

In [286]:
from sklearn.ensemble import RandomForestRegressor

In [287]:
rf_mae_scores = fit_model(RandomForestRegressor(), X_train_scaled, y_train.ravel())

In [288]:
print(f"RF CV MAE scores = {rf_mae_scores}")

RF CV MAE scores = [0.42933537 0.36515146 0.34072268]


#### Model 3 - AdaBoost Regressor

In [289]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor

In [294]:
ada_mae_scores = fit_model(AdaBoostRegressor(), X_train_scaled, y_train.ravel())

In [295]:
ada_mae_scores

array([0.42621159, 0.39404671, 0.37053581])

### Create the entity

In [296]:
from dataclasses import dataclass
from pathlib import Path

In [297]:
@dataclass(frozen=True)
class ModelTrainerConfig:
    
    root_dir: Path
    x_train_path: Path
    y_train_path: Path
    model_name: str

### Create configuration manager

In [298]:
from src.StarbucksProject.constants import *
from src.StarbucksProject.utils.common import read_yaml, create_directories

In [299]:
class ConfigurationManager:

    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH, 
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        
        config = self.config.model_training

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            x_train_path=config.x_train_path,
            y_train_path=config.y_train_path,
            model_name=config.model_name)

        return model_trainer_config

### Create the component

In [300]:
from src.StarbucksProject import logger
import joblib

In [301]:
class ModelTrainer:
    
    def __init__(self, config: ModelTrainerConfig):
        
        self.config = config

    def train(self):
                
        X_train = pd.read_csv(self.config.x_train_path)
        y_train = pd.read_csv(self.config.y_train_path)
        
        
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
            
        knr = KNeighborsRegressor()
        knr.fit(X_train_scaled, y_train)
        
        joblib.dump(knr, os.path.join(self.config.root_dir, self.config.model_name))

### Create the pipeline

In [302]:
try:

    config = ConfigurationManager()

    model_trainer_config = config.get_model_trainer_config()

    model_trainer = ModelTrainer(config=model_trainer_config)

    model_trainer.train()
    
except Exception as e:

    raise e

[2024-07-12 14:00:00,412: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-07-12 14:00:00,414: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-12 14:00:00,417: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-07-12 14:00:00,418: INFO: common: created directory at: artifacts]
[2024-07-12 14:00:00,419: INFO: common: created directory at: artifacts/model_training]


### Modularize the code (once working)

- Return to project folder